# Basic CSGO analysis
#### Last Updated: June 15, 2020
The csgo package was developed with easy analysis in mind. To that end, the data parsed goes directly into Pandas dataframes, as shown in the first example notebook, [Parsing a CSGO demofile](https://github.com/pnxenopoulos/csgo/blob/master/examples/00_Parsing_a_CSGO_demofile.ipynb). Such a format allows for easy analysis, especially when using basic aggregate statistics.

### What statistics are common in CSGO?
CSGO uses a variety of statistics we can calculate from the demofile data. Some basic statistics are:

- Kill/Death Ratio (KDR), defined as $\frac{Kills}{Deaths}$.
- Average Damage per Round (ADR), defined as $\frac{Damage}{Rounds}$. 
- Headshot Percentage, defined as $\frac{Headshot Kills}{Kills}$.
- Utility damage, which just represents the amount of damage a player inflicts using incendiary, molotiv and HE grenades.

These are just a few basic statistics that can easily be calculate using our data. To start, we reference the same [demofile](https://www.hltv.org/matches/2344822/og-vs-natus-vincere-blast-premier-fall-series-2020) from the first notebook, where we look at the first map of the series, `de_dust2`.

In [1]:
from csgo.parser import DemoParser

# Create parser object
# Set log=True above if you want to produce a logfile for the parser
demo_parser = DemoParser(demofile = "og-vs-natus-vincere-m1-dust2.dem", demo_id = "OG-NaVi-BLAST2020", parse_rate=128)


# Parse the demofile, output results to dictionary
data = demo_parser.parse()
data_df = demo_parser.parse(return_type="df")

23:23:03 [INFO] Go version>=1.14.0
23:23:03 [INFO] Initialized CSGODemoParser with demofile og-vs-natus-vincere-m1-dust2.dem
23:23:03 [INFO] Setting demo id to OG-NaVi-BLAST2020
23:23:03 [INFO] Setting parse rate to 128
23:23:03 [INFO] Running Golang parser from /home/peter/.pyenv/versions/3.8.4/lib/python3.8/site-packages/csgo-0.1-py3.8.egg/csgo/parser/
23:23:03 [INFO] Looking for file at /home/peter/Downloads/csgo_notebooks/og-vs-natus-vincere-m1-dust2.dem
23:23:23 [INFO] Wrote demo parse output to OG-NaVi-BLAST2020.json
23:23:23 [INFO] Reading in JSON from OG-NaVi-BLAST2020.json
23:23:24 [INFO] JSON data loaded, available in the `json` attribute to parser
23:23:24 [INFO] Successfully parsed JSON output
23:23:24 [INFO] Successfully returned JSON output
23:23:24 [INFO] Running Golang parser from /home/peter/.pyenv/versions/3.8.4/lib/python3.8/site-packages/csgo-0.1-py3.8.egg/csgo/parser/
23:23:24 [INFO] Looking for file at /home/peter/Downloads/csgo_notebooks/og-vs-natus-vincere-m1-du

## KDR
KDR can be calculated through simple aggregations. In the `Kills` dataframe, accessed via `data["Kills"]`, we can simply tabulate the number of times a player appeared as the _Attacker_ (kills) and the _Victim_ (deaths).

In [2]:
kills = data_df["Kills"]
killers = kills.groupby(["AttackerName"]).size().reset_index(name="Kills")
deaths = kills.groupby(["VictimName"]).size().reset_index(name="Deaths")
kdr = killers.merge(deaths, left_on = "AttackerName", right_on = "VictimName")
kdr["KDR"] = kdr["Kills"]/kdr["Deaths"]
kdr = kdr[["AttackerName", "Kills", "Deaths", "KDR"]]
kdr.columns = ["PlayerName", "Kills", "Deaths", "KDR"]
kdr.sort_values(by=["KDR"], ascending=False)

,PlayerName,Kills,Deaths,KDR
7,mantuu,24,10,2.400000
3,NBK-,22,16,1.375000
8,s1mple,21,16,1.312500
0,Aleksib,17,16,1.062500
2,ISSAA,13,15,0.866667
5,electronic,15,19,0.789474
6,flamie,15,19,0.789474
9,valde,12,17,0.705882
1,Boombl4,12,18,0.666667
4,Perfecto,11,17,0.647059


## ADR
ADR can be calculate simply by taking the total damage produced by a player divided by the number of rounds. In CSGO, a player can inflict healthpoint (HP) or armor damage. Additionally, we provide the raw damage output in the column `HpDamage`, since weapons like the AWP can inflict damages of over 100, along with the normalized damages in `KillHpDamage`, which have a maximum value of 100, which is a player's total HP.

First, we find the total number of rounds played by checking the length of `data["GameRounds"]`. Then, for each damage entry, we sum the Hp and Armor damages. Then, we sum the total damage for each player and divide by the total number of rounds.

In [3]:
# Find total number of rounds
total_rounds = len(data["GameRounds"])
damages = data_df["Damages"]

# Add Armor damage to the HP damage for each damage entry to get total damage
damages["HpDamageArmor"] = damages["HpDamage"] + damages["ArmorDamage"]
damages["KillHpDamageArmor"] = damages["KillHpDamage"] + damages["ArmorDamage"]

# Calculate
adr = (damages.groupby(["AttackerName"])["HpDamageArmor", "KillHpDamageArmor"].sum()/total_rounds).reset_index()
adr.columns = ["PlayerName", "RawADR", "NormADR"]
adr.sort_values(by=["RawADR"], ascending=False)

<ipython-input-3-3c116be82b87>:10: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  adr = (damages.groupby(["AttackerName"])["HpDamageArmor", "KillHpDamageArmor"].sum()/total_rounds).reset_index()


,PlayerName,RawADR,NormADR
3,NBK-,141.00,120.12
8,s1mple,125.16,87.44
7,mantuu,124.56,90.64
0,Aleksib,110.44,92.84
6,flamie,110.16,72.44
5,electronic,95.80,79.08
1,Boombl4,94.28,77.80
2,ISSAA,80.24,60.92
4,Perfecto,78.24,67.04
9,valde,78.12,71.32


## Headshot Percentage
Headshots can be a measure of a player's aiming accuracy. In the kills dataframe, we indicate whether a kill was a headshot in the `IsHeadshot` column.

In [4]:
kills.groupby("AttackerName").IsHeadshot.mean().reset_index(name="HeadShotPct").sort_values("HeadShotPct", ascending=False)

,AttackerName,HeadShotPct
6,flamie,0.733333
2,ISSAA,0.615385
0,Aleksib,0.588235
1,Boombl4,0.500000
5,electronic,0.466667
9,valde,0.416667
3,NBK-,0.409091
4,Perfecto,0.363636
7,mantuu,0.333333
8,s1mple,0.285714


We can also break down each player's headshot percentage by the weapon type. We filter by player-weapon combos that had at minimum 3 kills.

In [5]:
headshots = kills.groupby(["AttackerName", "Weapon"]).IsHeadshot.agg(["mean", "count"]).reset_index()
headshots = headshots[headshots["count"] > 3]
headshots.sort_values("mean", ascending=False)

,AttackerName,Weapon,mean,count
46,mantuu,USP-S,0.750000,4
54,valde,AK-47,0.750000,4
0,Aleksib,AK-47,0.666667,9
12,ISSAA,AK-47,0.666667,6
32,flamie,AK-47,0.600000,5
50,s1mple,Desert Eagle,0.500000,4
6,Boombl4,AK-47,0.400000,5
29,electronic,M4A4,0.400000,5
17,NBK-,AK-47,0.333333,9
21,NBK-,M4A4,0.333333,6


## Utility Damage
Utility damage, inflicted by grenades such as the incendiary grenade (on CT), molotov (on T) and HE grenade, can be another measure of player skill. While smokes and flash grenades can injure opponents (while this is rare, it exists in our selected data!) we do not count these damage events as utility damage. We can access grenade damage events in the damages dataframe, and the grenade events in the grenades dataframe.

In [6]:
damages.Weapon.unique()

array(['USP-S', 'Glock-18', 'p250', 'Tec-9', 'FAMAS', 'MP9',
       'Desert Eagle', 'M4A4', 'HE Grenade', 'AK-47', 'SSG 08', 'Molotov',
       'MAC-10', 'Incendiary Grenade', 'AWP', 'AUG', 'World', 'Galil AR',
       'C4', 'UMP-45', 'CZ75 Auto', 'Flashbang'], dtype=object)

In [7]:
nade_dmg = damages[damages["Weapon"].isin(["Incendiary Grenade", "Molotov", "HE Grenade"])]
nade_dmg = nade_dmg.replace("Incendiary Grenade", "Incendiary")
nade_dmg = nade_dmg.replace("Molotov", "Incendiary")
nade_dmg_df = nade_dmg.groupby("AttackerName").HpDamageArmor.sum().reset_index(name="UtilityDamage").sort_values("UtilityDamage", ascending=False)
nade_dmg_df.columns = ["PlayerName", "UtilityDamage"]
nade_dmg_df

,PlayerName,UtilityDamage
0,Aleksib,317
8,s1mple,202
9,valde,200
6,flamie,176
2,ISSAA,175
7,mantuu,64
3,NBK-,51
5,electronic,48
4,Perfecto,40
1,Boombl4,34


We can also break down the utilty damage by the associated grenade.

In [8]:
nade_dmg.groupby(["AttackerName", "Weapon"]).HpDamageArmor.sum().reset_index(name="UtilityDamage").sort_values("UtilityDamage", ascending=False)

,AttackerName,Weapon,UtilityDamage
0,Aleksib,HE Grenade,210
10,flamie,HE Grenade,155
16,valde,HE Grenade,154
15,s1mple,Incendiary,136
3,ISSAA,HE Grenade,122
1,Aleksib,Incendiary,107
14,s1mple,HE Grenade,66
4,ISSAA,Incendiary,53
9,electronic,Incendiary,48
6,NBK-,Incendiary,48


To find out how many grenades a player threw, we can access the grenades dataframe.

In [9]:
grenades = data_df["Grenades"]
nades_thrown = grenades[grenades["GrenadeType"].isin(["HE Grenade", "Incendiary Grenade", "Molotov"])].groupby("PlayerName").size().reset_index(name="NadesThrown")
nades_thrown

,PlayerName,NadesThrown
0,Aleksib,24
1,Boombl4,19
2,ISSAA,17
3,NBK-,17
4,Perfecto,18
5,electronic,12
6,flamie,15
7,mantuu,14
8,s1mple,7
9,valde,19


Lastly, we can combine `nade_dmg` and `nades_thrown` to create a dmg-per-nade metric.

In [10]:
nade_df = nade_dmg_df.merge(nades_thrown, on = "PlayerName")
nade_df["DmgPerNade"] = nade_df["UtilityDamage"]/nade_df["NadesThrown"]
nade_df.sort_values("DmgPerNade", ascending=False)

,PlayerName,UtilityDamage,NadesThrown,DmgPerNade
1,s1mple,202,7,28.857143
0,Aleksib,317,24,13.208333
3,flamie,176,15,11.733333
2,valde,200,19,10.526316
4,ISSAA,175,17,10.294118
5,mantuu,64,14,4.571429
7,electronic,48,12,4.000000
6,NBK-,51,17,3.000000
8,Perfecto,40,18,2.222222
9,Boombl4,34,19,1.789474
